In [ ]:
import numba as nb
from matplotlib import patches

## Radial Integration

In [ ]:
def calculatePsi(a0, a1, b0, b1, r):
    # TODO: Check if r^2 can be factored?
    L0 = np.sqrt(r*r - a0*a0)
    L1 = np.sqrt(r*r - a1*a1)
    theta0 = r*r * np.arccos(a0/r)
    theta1 = r*r * np.arccos(a1/r)
    psi1 = L0*a0 - L1*a1 + (b1-b0)*r*r + theta1 - theta0
    psi2 = (a0*L0-a1*L1)*r + a0*a0*a0*np.log(2*L0+2*r) - a1*a1*a1*log(2*L1+2*r) + 2*r*r*r*(b1-b0)+2*r*(theta1-theta0)
    return psi1, psi2

def calculatePhi(x, h):
    L = np.sqrt(1-x*x)
    theta = np.arcsin(x)
    phi1 = L*x + theta + x * h
    phi2 = 2*L*x*x - 2*L - 3*x*x*h
    phi3 = 3*x*(1+h*h) - x*x*x - 3*h*(L*x-theta)
    return phi1, phi2, phi3

def polarizeLine(x0, y0, x1, y1, r0=None):
    if r0 is None:
        r0 = np.sqrt(x0*x0 + y0*y0)
    theta0 = np.arctan2(y0, x0)
    b = np.arctan2(y1-y0, x1-x0) - np.pi/2
    a = r0*np.cos(theta0 - b)
    if a < 0:
        return -a, -b
    return a, b

def circleTransform(x0, y0, x1, y1, a, b):
    dx = (x1-x0)/a
    dy = (y1-y0)/b
    L = np.sqrt(dx*dx + dy*dy)
    h = np.sqrt(1 - L*L)
    return L, h

# calculatePhi(1, .8)
# calculatePsi(0, 0.3, 1, 2.1, .7)
# print(polarizeLine(-1, 1, 1, 1))

In [ ]:
a = 15.4
b = 2.3

t = linspace(0, 2*pi, 200)
xe = a*cos(t)
ye = b*sin(t)

tt = -np.array([.35, 1.33])
pt = np.column_stack([a*cos(tt), b*sin(tt)])

In [ ]:
L, h = circleTransform(pt[0,0], pt[0,1], pt[1,0], pt[1,1], a, b)
print(L,h)

w = 0.8
xp, yp = pt[0]*(1-w) + pt[1]*w
print(xp, yp)

# TODO: Fix sign issues
up1 = 2 * L * (0.5 - (xp - pt[0,0])/(pt[1,0] - pt[0,0]))
up2 = 2 * L * (0.5 - (yp - pt[0,1])/(pt[1,1] - pt[0,1]))
print(up1, up2)

In [ ]:
fig, ax = subplots(1, 2, figsize=(12,4))
ax[0].plot(xe,ye)
ax[0].plot(pt[:,0], pt[:,1])
ax[0].plot(xp,yp, '+')
ax[0].set_aspect(1)
ax[1].plot(cos(t), sin(t)-h)
ax[1].plot([-L, L], [0,0])
ax[1].plot(up1, 0, "x")
ax[1].plot(up2, 0, "+")
ax[1].set(ylim=[0.2*(h-1),1.2*(1-h)], xlim=(-1.2*L,1.2*L))

In [ ]:
def transitIntegral(a, b, x0, y0, dt=np.pi/5, dr=0.01, debug=None):
    if debug is not None:
        t = linspace(0,2*pi,100)
        debug.plot(cos(t), sin(t))
        debug.plot(a*cos(t)+x0, b*sin(t)+y0)
        
    # Check if the ellipse contains the origin
    if (x0/a)**2 + (y0/b)**2 < 1:
        raise NotImplementedError
        
    # Find the point on the ellipse closest to the origin
    loss = lambda t: (a*cos(t)+x0)**2 + (b*sin(t)+y0)**2
    closest = optimize.minimize_scalar(loss, bounds=[0,2*pi])
    if closest.fun >= 1:
        return 0.
    
    loss2 = lambda t: -(a*cos(t)+x0)**2 - (b*sin(t)+y0)**2
    furthest = optimize.minimize_scalar(loss2, bounds=[0,2*pi])
    
    # Find intersection points or furthest point (integral stopping condition)
    loss3 = lambda t: (a*cos(t)+x0)**2 + (b*sin(t)+y0)**2 - 1
    lStop = rStop = furthest.x
    if furthest.fun < -1:
        # TODO: Fix these bounds for when you're at the end of ingressing
        lStop = optimize.brentq(loss3, closest.x-np.pi, closest.x)
        rStop = optimize.brentq(loss3, closest.x, closest.x+np.pi)
    if debug is not None:
        debug.plot(a*cos(closest.x)+x0, b*sin(closest.x)+y0, '.')
        debug.plot(a*cos(furthest.x)+x0, b*sin(furthest.x)+y0, 'o')
        debug.plot(a*cos(lStop)+x0, b*sin(lStop)+y0, '+')
        debug.plot(a*cos(rStop)+x0, b*sin(rStop)+y0, '+')
    
    # Setup and integrate radially outward
    r0 = r1 = np.sqrt(closest.fun)
    lt = rt = closest.x
    lx0 = lx1 = rx0 = rx1 = a*np.cos(closest.x)+x0
    ly0 = ly1 = ry0 = ry1 = b*np.sin(closest.x)+y0
    lr1 = rr1 = r0
    while r1 < 1-0.01*dr:
        # print(f"{r0:10.3} {lr1:10.3} {rr1:10.3}")
        r0 = r1
        if np.isclose(r0, lr1):
            # Expand the left bounding line
            lt = max(lt-dt, lStop)
            lx0, ly0 = lx1, ly1
            lx1, ly1 = a*np.cos(lt)+x0, b*np.sin(lt)+y0
            lr1 = np.sqrt(lx1*lx1 + ly1*ly1)
            debug.plot([lx0, lx1], [ly0, ly1], color="grey")
        if np.isclose(r0, rr1):
            # Expand the right bounding line
            rt = min(rt+dt, rStop)
            rx0, ry0 = rx1, ry1
            rx1, ry1 = a*np.cos(rt)+x0, b*np.sin(rt)+y0
            rr1 = np.sqrt(rx1*rx1 + ry1*ry1)
            debug.plot([rx0, rx1], [ry0, ry1], color="silver")
        r1 = min(r0+dr, rr1, lr1)
        
        # Plot the new circle
        loss4 = lambda u: (lx0*(1-u)+lx1*u)**2 + (ly0*(1-u)+ly1*u)**2 - r1*r1
        uli = optimize.brentq(loss4, 0, 1+1e-10)
        loss5 = lambda u: (rx0*(1-u)+rx1*u)**2 + (ry0*(1-u)+ry1*u)**2 - r1*r1
        uri = optimize.brentq(loss5, 0, 1+1e-10)
        
        thetaStart = np.arctan2(ly0*(1-uli)+ly1*uli, lx0*(1-uli)+lx1*uli)
        thetaStop = np.arctan2(ry0*(1-uri)+ry1*uri, rx0*(1-uri)+rx1*uri)
        tt = np.linspace(thetaStart, thetaStop, 100)
        debx, deby= r1*cos(tt), r1*sin(tt)
        debug.plot(debx, deby, color="orange", alpha=.5)
        
    return

fig, ax = subplots(1,1)
ax.set_aspect(1)
transitIntegral(.3, .2, -.5, .75, debug=ax, dr=.01)
gca().set(xlim=(-.85,-.15), ylim=(0.5,1));

In [ ]:
from scipy.signal import argrelextrema

In [ ]:
a = 1
b = .5
r = .001
xe, ye = -.63,.05

t = linspace(-1,7,500)
fig, ax = subplots(1,2, figsize=(10,4))

ax[0].plot(xe, ye, '.')
ax[0].plot(a*cos(t), b*sin(t))

ax[0].set(xlim=(-1.1,1.1), ylim=(-1.1,1.1))
ax[0].axvline(0., color="silver", alpha=.2)
ax[0].axhline(0., color="silver", alpha=.2)

loss = lambda t: (a*cos(t)-xe)**2 + (b*sin(t)-ye)**2
ax[1].plot(t/(2*pi), loss(t))
ax[1].axvline(0.5, color="silver", alpha=.2)
ax[1].set(xlim=(0,1))

# Get Extrema
tmin = t[argrelextrema(loss(t),np.less)]
tmax = t[argrelextrema(loss(t),np.greater)]
ax[0].plot(a*cos(tmin), b*sin(tmin), 'x')
ax[0].plot(a*cos(tmax), b*sin(tmax), 'o')

Procedure:
1. Locate nearest and furthest points, return 0 if no overlap
2. If ellipse contains circle center
    1. TODO.
    2. iterate in r from rNear to min(rFar, 1) according to step size.  Can exclude the actual boundaries (l=0 by definition)

In [ ]:
# TODO: Optimize
def locateIntersection(a, b, r, x0, y0):
    '''Finds the intersections of an axis-aligned ellipse at the origin with radii
    a and b with a circle at x0, y0 and radius r. Returns:
        - nan, 0, 0, 0 if the shapes do not intersect at all
        - nan, 1, 1, 1 if the ellipse is fully contained by the circle
        - x1, y1, x2, y2 if the shapes intersect (the intersection coordinates)'''
    loss = lambda t: (a*sin(t)-x0)**2 + (b*cos(t)-y0)**2
    closest = optimize.minimize_scalar(loss, bounds=[0,2*pi])
    if closest.fun >= r**2:
        # Shapes do not intersect
        return np.nan, 0, 0, 0
    if loss(closest.x+np.pi) <= r**2:
        # Ellipse is fully inside the circle
        return np.nan, 1, 1, 1
    dist = lambda t: (a*sin(t)-x0)**2 + (b*cos(t)-y0)**2 - r**2
    root1 = optimize.brentq(dist, closest.x, closest.x+np.pi)
    root2 = optimize.brentq(dist, closest.x, closest.x-np.pi)
    return a*sin(root1), b*cos(root1), a*sin(root2), b*cos(root2)

%timeit locateIntersection(3, 2, 10., 7, 0)
locateIntersection(3, 2, 10., 7, 0)

In [ ]:
@nb.njit(fastmath=True)
def bisect2d(x0, x1, y, zTarget, grid, tol=1e-5):
    i = 0
    z0 = nt.interp2d(grid, x0, y)
    z1 = nt.interp2d(grid, x1, y)
    while np.abs(x1 - x0) > tol and i < 100:
        i += 1
        xNew = nt.clip((zTarget - z0) / (z1-z0), 0.15, 0.85)
        if i%2 == 1:
            xNew = x0 + 1.1 * (x1-x0) * xNew
        else:
            xNew = x0 + 0.9 * (x1-x0) * xNew
        zNew = nt.interp2d(grid, xNew, y)
        if zNew < zTarget:
            x0 = xNew
            z0 = zNew
        else:
            x1 = xNew
            z1 = zNew
    return (x1+x0)/2.

## Bisecting Triangles

In [ ]:
# TODO: Optimize
def locateIntersection(a, b, r, x0, y0):
    '''Finds the intersections of an axis-aligned ellipse at the origin with radii
    a and b with a circle at x0, y0 and radius r. Returns:
        - nan, 0, 0, 0 if the shapes do not intersect at all
        - nan, 1, 1, 1 if the ellipse is fully contained by the circle
        - x1, y1, x2, y2 if the shapes intersect (the intersection coordinates)'''
    loss = lambda t: (a*sin(t)-x0)**2 + (b*cos(t)-y0)**2
    closest = optimize.minimize_scalar(loss, bounds=[0,2*pi])
    if closest.fun >= r**2:
        # Shapes do not intersect
        return np.nan, 0, 0, 0
    if loss(closest.x+np.pi) <= r**2:
        # Ellipse is fully inside the circle
        return np.nan, 1, 1, 1
    dist = lambda t: (a*sin(t)-x0)**2 + (b*cos(t)-y0)**2 - r**2
    root1 = optimize.brentq(dist, closest.x, closest.x+np.pi)
    root2 = optimize.brentq(dist, closest.x, closest.x-np.pi)
    return a*sin(root1), b*cos(root1), a*sin(root2), b*cos(root2)

%timeit locateIntersection(3, 2, 10., 7, 0)
locateIntersection(3, 2, 10., 7, 0)

In [ ]:
def transitIntegral(a, b, r, xc, yc, debug=None, edgeTol=1e-2):
    # if debug is not None:
        # debug.add_patch(patches.Ellipse([0,0], 2*a, 2*b, ec=(.5, .7, 1., 1.), fc=(.5, .7, 1., 1.)))
        # debug.add_patch(patches.Circle([xc, yc], r, ec=(1,1,0,0.5), fc=(1,1,0,.2)))
        
    total = 0.
    stack = np.zeros((64, 4))
    xi1, yi1, xi2, yi2 = locateIntersection(a, b, r, xc, yc)
    # if debug is not None:
        # plot([xi1, xi2], [yi1, yi2], color="Black", ls="dotted")
    
    if np.isnan(xi1):
        if yi1 == 0:
            print("Out of transit")
            return 0.
        if yi1 == 1:
            print("Fully in-transit")
            stack[0] = 0, 1, sqrt(.5), -sqrt(.5)
            total += integrateChord(stack, None, edgeTol) * a*b
            stack[0] = sqrt(.5), -sqrt(.5), 0, 1
            total += integrateChord(stack, debug, edgeTol) * a*b
    else:
        # Integrate towards the star in star-centered unit-sphere coordinates
        stack[0] = (xi2-xc)/r, (yi2-yc)/r, (xi1-xc)/r, (yi1-yc)/r
        total += integrateChord(stack, None, edgeTol) * r*r
        # Integrate towards the planet
        stack[0] = xi1/a, yi1/b, xi2/a, yi2/b
        total += integrateChord(stack, debug, edgeTol) * a*b
    return total

def integrateChord(stack, debug=None, edgeTol=0.1):
    i = 0
    total = 0.
    if debug is not None:
        debug.plot(stack[i,::2], stack[i,1::2], color="Grey", ls="solid", zorder=10)
    
    while i >= 0:
        x0, y0, x1, y1 = stack[i]
        # Calculate line that bisects the line segment at the top of the stack
        xBisect = 0.5*(x0+x1)
        yBisect = 0.5*(y0+y1)
        mBisect = 0. if (y0 == y1) else -(x1-x0)/(y1-y0)
        bBisect = yBisect - xBisect * mBisect  # TODO: Handle vertical case
        
        xt = linspace(-10,10,100)
        yt = xt*mBisect + bBisect
        # debug.plot(xt*r + xc, yt*r + yc)
        
        # Get intersection of this line with the planet edge (towards the star)
        xnew = (-bBisect * mBisect - np.sign(y1-y0)* np.sqrt(mBisect*mBisect - bBisect*bBisect + 1.)) / (mBisect*mBisect + 1)
        ynew = mBisect * xnew + bBisect
        # debug.plot(xc+xnew*r, yc+ynew*r, 'o')
        if debug is not None:
            debug.plot(np.array([x0, x1, xnew, x0]), [y0, y1, ynew,y0], ls="dashed")
        
        # Evaluate error
        distsq = (xBisect-xnew)**2 + (yBisect-ynew)**2
        # print(f"{i}    {x0:10.3} {y0:10.3}    {x1:10.3}, {y1:10.3}     {distsq:10.4}    {distsq < edgeTol**2}")
        if distsq < edgeTol**2 or i >= 32:
            # Within tol: external integral. TODO: better external integral
            i -= 1
            total += np.sqrt(distsq * ((x1-x0)**2 + (y1-y0)**2)) * 2/3
        else:
            # Outside tol: integrate enclosed triangles, push external to stack.
            total += np.sqrt(distsq * ((x1-x0)**2 + (y1-y0)**2)) / 2.
            stack[i] = xnew, ynew, x1, y1 
            stack[i+1] = x0, y0, xnew, ynew
            i += 1
    return total


a, b, r = 3, 2, 7.5
xc, yc = 4, 4
fig, axes = subplots(2, 2, figsize=(8,8), sharex=True, sharey=True)
for i, ax in enumerate(axes.flatten()):
    ax.set(xlim=(-1, 1), ylim=(-1, 1))
    print(transitIntegral(a, b, r, xc*(-1)**i, -yc*(-1)**(i//2), debug=ax, edgeTol=1e-3) / a / b - pi)
    # ax.set(xlim=(-1.2*a, 1.2*a), ylim=(-1.2*b, 1.2*b))

In [ ]:
%time transitIntegral(a, b, r, 0, 0, edgeTol=1e-3)/a/b - pi

## Python Integration

In [ ]:
def transitIntegral(a, b, x0, y0):
    xmin = max(-a, x0-1)
    xmax = min(a, x0+1)

    if x0*x0 + y0*y0 > (1+b)**2 or xmin==xmax:
        return 0.
    
    def ymin(x):
        ellipseMin = -b*np.sqrt(1-(x/a)**2)
        circleMax = y0+np.sqrt(1 - (x-x0)**2)
        circleMin = y0-np.sqrt(1 - (x-x0)**2)
        return min(max(ellipseMin, circleMin), circleMax)
    
    def ymax(x):
        ellipseMax = b*np.sqrt(1-(x/a)**2)
        circleMax = y0+np.sqrt(1 - (x-x0)**2)
        circleMin = y0-np.sqrt(1 - (x-x0)**2)
        return max(min(ellipseMax, circleMax), circleMin)
    
    @nb.njit
    def integrand(x, y):
        c = sqrt(x**2 + y**2)
        return 0.3 + .93 * c -.23*c*c
    
    # print(max(-a, x0-1), min(a, x0+1))
    # xt = np.linspace(max(-a, x0-0.5), min(a, x0+0.5), 100)
    # print(np.array([(xti, ymin(xti) - ymax(xti)) for xti in xt]))
    return dblquad(func=integrand, a=xmin, b=xmax, gfun=ymin, hfun=ymax)[0]
    # return dblquad(func=integrand, a=-a, b=a, gfun=ymin, hfun=ymax)
    
%time transitIntegral(.15, .13, -1, 0)

## Triangle Integration

In [ ]:
import numba as nb
from numpy.linalg import norm
from scipy.integrate import dblquad

In [ ]:
@nb.njit(inline="always")
def integrand(x, y, params):
    # return x**2+y**2
    # return y
    # return x**4 - params[0]*x + params[1]*y**2 + params[2]
    return 1.0
    # return sin(y)
    # return y**2

@nb.njit(fastmath=True, inline="always")
def dunavant7(newF, x1, y1, f1, x2, y2, f2, x3, y3, f3, params):
    total = (f1+f2+f3)/40. + newF/15.
    total += integrand((x1+x2)/2., (y1+y2)/2, params) / 15.
    total += integrand((x1+x3)/2., (y1+y3)/2, params) / 15.
    total += integrand((x1+x2+x3)/3., (y1+y2+y3)/3, params) * (9./40.)
    jacobian = (x2-x1)*(y3-y1) - (y2-y1)*(x3-x1)
    return total * abs(jacobian)

In [ ]:
@nb.njit(fastmath=True)
def integrateTriangle(stack, params, tol=1e-3, initEval=True):
    '''Stack is a 16 array, with entry 0 containing the right triangle to integrate,
    defined by 3 x-y pairs and their function values starting with the right angled one.'''
    # xyf: 0,1,2     3,4,5    6,7,8
    
    if initEval:
        stack[0,2] = integrand(stack[0,0], stack[0,1], params)
        stack[0,5] = integrand(stack[0,3], stack[0,4], params)
        stack[0,8] = integrand(stack[0,6], stack[0,7], params)

    i = 0
    total = 0.
    while i >= 0:
        x0, y0, f0, x1, y1, f1, x2, y2, f2 = stack[i]
        newX = (x1+x2)/2.
        newY = (y1+y2)/2.
        newF = integrand(newX, newY, params)
        if abs(newF - (f2+f1)/2.) < tol or i >= 12:
            total += dunavant7(newF, x0, y0, f0, x1, y1, f1, x2, y2, f2, params)
            i -= 1
        else:
            # Append two triangles subdivided from the first
            stack[i+1] = newX, newY, newF, x0, y0, f0, x1, y1, f1
            stack[i] = newX, newY, newF, x0, y0, f0, x2, y2, f2
            i += 1
            
    return total

def integrateDebugTriangle(stack, params, **args):
    assert stack[0,0] == stack[0,3], stack[0,0]
    def top(x):
        return (x-stack[0,0])*(stack[0,7]-stack[0,4])/(stack[0,6]-stack[0,3]) + stack[0,4]
    
    def bot(x):
        return (x-stack[0,0])*(stack[0,7]-stack[0,1])/(stack[0,6]-stack[0,0]) + stack[0,1]
    
    x = linspace(0,1,10)
    plot(x,top(x), color="black")
    plot(x,bot(x), color="orange")
    return dblquad(lambda x,y: integrand(y, x, params), stack[0,0], stack[0,6], bot, top, **args)[0]

In [ ]:
tol = 1e-3
stack = np.zeros((16, 9))
params = 1.*ones(3)
stack[0] = 0, -1, integrand(0,-1,params), 0, 1, integrand(0,1,params), 1, 0, integrand(1,0,params)

%time print(integrateDebugTriangle(stack, params))#, epsabs=1e-4, epsrel=1e-8))
%time print(integrateTriangle(stack, params, tol=tol))
stack[0] = 0, -1, integrand(0,-1, params), 0, 1, integrand(0,1, params), 1, 0, integrand(1,0, params)
print(integrateDebugTriangle(stack, params) - integrateTriangle(stack, params, tol=tol))

# Old Attempts

In [ ]:
def transitIntegral(a, b, r, xc, yc, debugPlot=None):
    if debugPlot is not None:
        debugPlot.add_patch(patches.Ellipse([0,0], 2*a, 2*b, ec=(.5, .7, 1., 1.), fc=(.5, .7, 1., 1.)))
        debugPlot.add_patch(patches.Circle([xc, yc], r, ec=(1,1,0,0.5), fc=(1,1,0,.2)))
        
    xi1, yi1, xi2, yi2 = locateIntersection(a, b, r, xc, yc)
    if debugPlot is not None:
        print(xi1, yi1, xi2, yi2)
        plot([xi1, xi2], [yi1, yi2], '.')
    
    if np.isnan(xi1) and yi1 == 0:
        # Not transiting, can stop early
        print("out of transit")
        return 0.

    # Prepare the stack of rectangles: x1, y1, x2, y2, starEdge
    stack = np.zeros((64, 5))
    stack[0] = -a, -b, 0., 0., 0.
    stack[1] = -a, 0, 0., b, 0.
    stack[2] = 0, -b, a, 0., 0.
    stack[3] = 0., 0., a, b, 0.
    i = 3
    print(stack[:i+1])
    
    # Pre-subdivide at the intersections
    j = 0
    while j <= i and np.isfinite(xi1):
        x0, y0, x1, y1, starEdge = stack[j]
        if (x0 < xi1 < x1) and (y0 < yi1 < y1):
            stack[i+1] = x0, y0, xi1, yi1, 0.
            stack[i+2] = x0, yi1, xi1, y1, 0.
            stack[i+3] = xi1, y0, x1, yi1, 0.
            stack[j] = xi1, yi1, x1, y1, 0.
            i += 3
        elif (x0 < xi2 < x1) and (y0 < yi2 < y1):
            stack[i+1] = x0, y0, xi2, yi2, 0.
            stack[i+2] = x0, yi2, xi2, y1, 0.
            stack[i+3] = xi2, y0, x1, yi2, 0.
            stack[j] = xi2, yi2, x1, y1, 0.
            i += 3
        else:
            j += 1
    
    print(i)
    print(stack[:i+1])
    if debugPlot is not None:
        for j in range(i+1):
            debugPlot.add_patch(Rectangle(stack[j,:2], stack[j,2]-stack[j,0], stack[j,3]-stack[j,1], fc=(0,0,0,0), ec=(0,0,0,1)))
        
    # Detect partial overlaps and subdivide
    # Figure out whether each cell contains a planet or star boundary (or none)
        
        
    return None


a, b, r = 3, 2, 7.5
xc, yc = 9, -3

ax = axes(aspect='equal')
xlim(-1.2*a, 1.2*a)
ylim(-1.2*b, 1.2*b)
transitIntegral(a, b, r, xc, yc, debugPlot=ax)
savefig("/home/danielpt/normal.png")

## Circle Tests

In [ ]:
@nb.njit
def integrateCircle(x0, y0, x1, y1, tol=.01, it=-25):
    # Is the box inside, outside, or crossing the edge of the sphere?
    # Shortcut since we know the orientation of the rect relative to the circle
    r0 = x0*x0 + y0*y0
    r1 = x1*x1 + y1*y1
    if r0 + 1e-12 >= 1:
        # Fully outside the circle
        return 0.
    elif r1 <= 1+1e-12:
        # Fully inside the circle
        return (x1-x0) * (y1-y0)
    else:
        # Crossing the edge of the circle (the interesting case)
        xc = 0.5*(x0 + x1)
        yc = np.sqrt(1 - xc**2)
        err = (yc-y0) / (y1-y0) - 0.5
        if abs(err) < tol or it >= 0:
            return ((y1-y0) + 4*(yc-y0)) * (x1-x0) / 6.
        tot = (xc-x0) * (yc-y0)
        tot += integrateCircle(xc, y0, x1, yc, tol, it+1)
        tot += integrateCircle(x0, yc, xc, y1, tol, it+1)
        return tot


integrateCircle(0, 0, 1, 1, 1e-5, 0)
%time 4*integrateCircle(0, 0, 1, 1, 1e-4, -35) - pi

In [ ]:
1/2 - a/6

In [ ]:
pi/4.

In [ ]:
x = linspace(0,1,100)
y0 = sqrt(3)/2.
# a = (y0-.5)*4/3.
a = 3
b = -4*y0 - 5/2
c = 1/5 + 3*a/2 - b
f = lambda x: a*x*x*x + b*x*x + c*x + 1
print(a, f(0.5)/y0)
plot(x, f(x))
plot(x,1-x)

In [ ]:
@nb.njit
def integrateCircle(x0, y0, x1, y1, tol=.01, it=-25):
    # Is the box inside, outside, or crossing the edge of the sphere?
    # Shortcut since we know the orientation of the rect relative to the circle
    # print(f"Iteration {it}: {x0:.3f}, {y0:.3f}, {x1:.3f}, {y1:.3f}")
    r0 = x0*x0 + y0*y0
    r1 = x1*x1 + y1*y1
    if r0 + 1e-12 >= 1:   # Fully outside the circle
        # print("Terminal Outer")
        return 0.
    elif r1 <= 1+1e-12:  # Fully inside the circle
        # print("Terminal Inner")
        return (x1-x0) * (y1-y0)
    else:        # Crossing the edge of the circle (the interesting case)
        m = (y1 - y0) / (x1 - x0)
        b = y0 - m*x0
        # Assuming the + solution (valid here, not later)
        xc = (-m*b + np.sqrt(m*m - b*b + 1)) / (1 + m*m)
        yc = m*xc + b
        # print('\t', xc, yc)
        err = (xc-x0) / (x1-x0) - 0.5
        # print("\tErr", it, err, tol)
        if abs(err) < tol or it >= 0:
            # Return the triangle, for now TODO: parabola
            xc = 0.5*(x0 + x1)
            yc = np.sqrt(1 - xc**2)
            return ((y1-y0) + 4*(yc-y0)) * (x1-x0) / 6.
        tot = (xc-x0) * (yc-y0)
        tot += integrateCircle(xc, y0, x1, yc, tol, it+1)
        tot += integrateCircle(x0, yc, xc, y1, tol, it+1)
        return tot


integrateCircle(0, 0, 1, 1, 1e-3, -5)
%time 4*integrateCircle(0, 0, 1, 1, 1e-4, -25) - np.pi

## Algebra

In [ ]:
import sympy as sy
sy.init_printing(use_latex=True, fontsize=16)

x, y, d0, d1, a, b, c = sy.symbols('x y d_0, d_1 a b c', real=True)

In [ ]:
# Cubic function, f(0) = 1
f = a*x**3 + b *x**2 + d0*x + 1
f

In [ ]:
# f'(0) = d0
f = f.subs(b, sy.solve(f.diff(x).subs(x,1) - d1, b)[0]).simplify()
f

In [ ]:
# f(1) = 0
f = f.subs(a, sy.solve(f.subs(x, 1), a)[0]).simplify()
f

In [ ]:
f.integrate((x, 0, 1))

In [ ]:
center = -2
rad = sy.sqrt((1 - center)**2 + center**2)
circ = center + sy.sqrt(rad**2 - (x-center)**2)
f1 = f.subs(d0, circ.diff(x).subs(x,0))
f1 = f1.subs(d1, circ.diff(x).subs(x,1))
integral = circ.integrate((x,0,1)).evalf()
print(integral/.5-1.)
mid = circ.subs(x,0.5).evalf()
print((1 + 4*mid)/ (6*integral) - 1)
print(f1.integrate((x,0,1)).evalf() / integral-1.)
sy.plot(
    f1,
    circ,
    1 - x,
    (x, 0, 1))